In [25]:
import sys 
sys.path.append('..')
from dataset.vqav2 import VQADataset
dataset = VQADataset() # "/home/work/yuna/HPA/data/vqav2_val.json" 
len(dataset)

1000

In [33]:
dataset[0]

{'image_id': 'COCO_val2014_000000189936.jpg',
 'question_id': 189936002,
 'question_type': 'what is the woman',
 'question': 'Question: What is the woman in gray doing? Answer the question using a single word or phrase. \nAnswer:',
 'answers': [{'answer': 'taking pictures',
   'answer_confidence': 'yes',
   'answer_id': 1},
  {'answer': 'taking photo', 'answer_confidence': 'yes', 'answer_id': 2},
  {'answer': 'photographing', 'answer_confidence': 'yes', 'answer_id': 3},
  {'answer': 'taking pictures', 'answer_confidence': 'yes', 'answer_id': 4},
  {'answer': 'taking picture', 'answer_confidence': 'yes', 'answer_id': 5},
  {'answer': 'taking picture', 'answer_confidence': 'yes', 'answer_id': 6},
  {'answer': 'photographing', 'answer_confidence': 'yes', 'answer_id': 7},
  {'answer': 'taking picture', 'answer_confidence': 'yes', 'answer_id': 8},
  {'answer': 'photographing', 'answer_confidence': 'yes', 'answer_id': 9},
  {'answer': 'taking picture', 'answer_confidence': 'yes', 'answer_id'

In [44]:
# swift_dataset(dataset)
swift_dataset(dataset, 'vlm')
swift_dataset(dataset, 'vlm', False)

saved to /home/work/yuna/HPA/data/swift/vlm_vqav2_val_1k_blind.jsonl
saved to /home/work/yuna/HPA/data/swift/vlm_vqav2_val_1k.jsonl


In [43]:
image_folder="/home/work/yuna/VLMEval/data/val2014"

In [42]:
def swift_dataset(vqa_data, models='llm', blind=True): 

    output_path=    f"/home/work/yuna/HPA/data/swift/{models}_vqav2_val_1k"
    if blind: 
        output_path += "_blind"
    output_path += ".jsonl"

    with open(output_path, 'w', encoding='utf-8') as f:
        for item in vqa_data:
            question = item['question']
            if not models == 'llm':
                question = f"<image>\n{question}"
            
            swift_item = {
                "id": str(item.get('question_id', item.get('image_id', ''))),
                "conversation": [
                    {
                        "from": "human",
                        "value": question
                    },
                    # {
                    #     "from": "assistant",
                    #     "value": item['multiple_choice_answer']
                    # }
                ]
            }
            if not models == 'llm':
                image_path = os.path.join(image_folder, item['image_id'])
                if blind: 
                    swift_item['image'] = ["/home/work/yuna/HPA/data/blank_224.png"]
                else: 
                   swift_item['image'] = [image_path]
            
            f.write(json.dumps(swift_item, ensure_ascii=False) + '\n')
    print(f"saved to {output_path}") 


In [19]:
modelscope_tsv(dataset, '/home/work/yuna/HPA/data/vqav2_1k/val.tsv')

Saved TSV with full VQA annotations to: /home/work/yuna/HPA/data/vqav2_1k/val.tsv


In [18]:
import csv
import os 
import json 

def modelscope_tsv(items, output_path, image_folder='/home/work/yuna/VLMEval/data/val2014'):
    """
    Write a list of items to a TSV file with columns:
    index, answer, question, image_path

    index	answer	question	        image_path
    1	Dog	What animal is this?	    /root/LMUData/images/custom_mcq/dog.jpg
    2	Museum	What building is this?	/root/LMUData/images/custom_mcq/AMNH.jpg
    3	Tokyo	Which city's skyline is this?	/root/LMUData/images/custom_mcq/tokyo.jpg
    4	Tesla	What is the brand of this car?	/root/LMUData/images/custom_mcq/tesla.jpg
    5	Running	What is the person in the picture doing?	/root/LMUData/images/custom_mcq/running.jpg
    
    Parameters:
        data: list of dicts, each containing:
              {
                  "answer": str,
                  "question": str,
                  "image_path": str
              }
        output_path: str, path to write .tsv file
    """
    fieldnames = [
        "index",
        "image_id",
        "question_id",
        "question_type",
        "answer_type",
        "question",
        "answer",
        "image_path",
    ]

    with open(output_path, "w", encoding="utf-8", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames, delimiter="\t")
        writer.writeheader()

        for idx, item in enumerate(items, start=1):
            writer.writerow({
                "index": idx,
                "image_id": item["image_id"],
                "question_id": item["question_id"],
                "question_type": item["question_type"],
                "answer_type": item["answer_type"],
                "question": item["question"],
                "answers": json.dumps(item["answers"], ensure_ascii=False),
                "multiple_choice_answer": json.dumps(item["multiple_choice_answer"], ensure_ascii=False),
                "image_path": os.path.join(image_folder, item["image_id"]),
            })

    print(f"Saved TSV with full VQA annotations to: {output_path}")
